In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

c:\Users\insung\LLM_RAG_PROJ\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 모델 및 토크나이저 로드
model_name = "aiqwe/FinShibainu"
tokenizer = AutoTokenizer.from_pretrained(model_name)


bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # 8-bit 양자화로 변경
    llm_int8_enable_fp32_cpu_offload=True
)

# 최적화된 device_map 설정
device_map = {
    "model.embed_tokens": 0,  # 임베딩은 GPU에서 실행
    "model.layers.0": 0,  # 첫 번째 레이어 GPU
    "model.layers.1": 0,  # 두 번째 레이어 GPU (가능하면)
    "model.layers.2": 0,  # 세 번째 레이어 GPU (가능하면)
    "model.layers.3": 0,  # 네 번째 레이어 GPU (가능하면)
    "model.layers.4": "cpu",  # VRAM 부족 대비하여 CPU로 배치
    "model.layers.5": "cpu",
    "model.layers.6": "cpu",
    "model.layers.7": "cpu",
    "model.layers.8": "cpu",
    "model.layers.9": "cpu",
    "model.layers.10": "cpu",
    "model.layers.11": "cpu",
    "model.layers.12": "cpu",
    "model.layers.13": "cpu",
    "model.layers.14": "cpu",
    "model.layers.15": "cpu",
    "model.layers.16": "cpu",
    "model.layers.17": "cpu",
    "model.layers.18": "cpu",
    "model.layers.19": "cpu",
    "model.layers.20": "cpu",
    "model.layers.21": "cpu",
    "model.layers.22": "cpu",
    "model.layers.23": "cpu",
    "model.layers.24": "cpu",
    "model.layers.25": "cpu",
    "model.layers.26": "disk",  # VRAM/CPU 메모리가 부족한 경우 디스크로 이동
    "model.layers.27": "disk",
    "model.norm": "cpu",  # LayerNorm은 CPU에서 실행
    "model.rotary_emb": "cpu",  # Rotary Embedding도 CPU에서 실행
    "lm_head": "cpu"  # 최종 출력 레이어는 CPU에서 실행
}

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
# 위와 같은 세팅으로 27초 가량 소요
# print(model.hf_device_map)


Loading checkpoint shards: 100%|██████████| 4/4 [00:25<00:00,  6.26s/it]
Some parameters are on the meta device because they were offloaded to the cpu and disk.


{'model.embed_tokens': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 'cpu', 'model.layers.5': 'cpu', 'model.layers.6': 'cpu', 'model.layers.7': 'cpu', 'model.layers.8': 'cpu', 'model.layers.9': 'cpu', 'model.layers.10': 'cpu', 'model.layers.11': 'cpu', 'model.layers.12': 'cpu', 'model.layers.13': 'cpu', 'model.layers.14': 'cpu', 'model.layers.15': 'cpu', 'model.layers.16': 'cpu', 'model.layers.17': 'cpu', 'model.layers.18': 'cpu', 'model.layers.19': 'cpu', 'model.layers.20': 'cpu', 'model.layers.21': 'cpu', 'model.layers.22': 'cpu', 'model.layers.23': 'cpu', 'model.layers.24': 'cpu', 'model.layers.25': 'cpu', 'model.layers.26': 'disk', 'model.layers.27': 'disk', 'model.norm': 'cpu', 'model.rotary_emb': 'cpu', 'lm_head': 'cpu'}


In [35]:
# 입력 예시
input_text = "금리가 높은 적금 상품 추천해줘"

# 입력을 토큰화
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
inputs = tokenizer(input_text, return_tensors="pt").to(device)


cuda


In [ ]:
# 모델 예측
with torch.no_grad():
    output = model.generate(**inputs, max_length=50)

# 결과 디코딩
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"💬 모델 응답: {response}")
#응답에 20분 가량 소요되었음..


💬 모델 응답: 금리가 높은 적금 상품 추천해줘요.금리가 높은 적금 상품을 찾고 계신다면, 여러 금융기관의 상품을 비교하고, 본인의 투자 성향과 목표에 맞는 상품을 선택하는 것이 중요합니다. 아래는 금리가 높은 적금 상품을 선택하는 데 도움이 될 수 있는 몇 가지 요


vLLM은 Lightweight LLM 프레임워크로 메모리 효율을 최적화 해주는 라이브러리\
해당 라이브러리를 사용하기 위해서는 window네이티브 환경에서는 불가능하고 wsl 환경에서 Ubuntu로 실행해야 함. 
\
하려고 했지만.. 로컬에서 VRAM이 너무 부족해 Vllm라이브러리를 사용할 수 없어 포기...

\
'''

from vllm import LLM
from vllm.sampling_params import SamplingParams

llm = LLM(model="aiqwe/FinShibainu", tensor_parallel_size=1)
sampling_params = SamplingParams(temperature=0.7, max_tokens=100)

inputs = [
    "금리가 높은 적금 상품은 왜 추천해줘."
]
outputs = llm.generate(inputs, sampling_params)

for o in outputs:
    print("질문:", o.prompt)
    print("생성된 답변:", o.outputs[0].text)
    print("*" * 100)


'''